<a href="https://colab.research.google.com/github/LeonidVM/tamagotchi/blob/main/ai_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -U langchain-community
!pip install gigachat
!pip install langgraph
!pip install langchain-openai
!pip install langchain-gigachat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.9/130.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.6 MB/s eta 0:00:00


In [4]:
from langchain.chat_models.gigachat import GigaChat
import pandas as pd
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver, InMemorySaver
import time
from typing import Dict
from langchain.tools import tool

In [5]:
auth = 'NWJkZTBiZTEtMjdkZC00ZTYyLWJhMjUtYThjOTJlZWVlYWYxOjc5YmUwMmI3LTVmYzctNDA1Mi04M2RjLTMzMzE5M2YwNjVhMQ=='
# secret = 'f070a059-ed42-4241-83e0-e6d96d9ea8e0'
# id = '5bde0be1-27dd-4e62-ba25-a8c92eeeeaf1'

In [8]:
loan_data_v1_df = pd.read_csv('knowledge_base/data_1.csv', sep=';')
loan_data_v1_df.loc[loan_data_v1_df['id'] == 1]

clint_data_df = pd.read_csv('knowledge_base/data_2_clients.csv', sep=';')
clint_data_df['fio'].loc[clint_data_df['id'] == 1]
loan_data_v1_df['fio'] = clint_data_df['fio']

In [9]:
@tool
def get_loan_data_v2(fio: str) -> Dict:
  '''
  Функция возвращает информацию о параметрах, которые были использованы при принятия решения о выдаче кредита

  Args:
    fio (str): ФИО клиента.

  Returns:
    Dict: Словарь с данными о параметрах, которые были использованы при принятия решения о выдаче кредита.
  '''
  return loan_data_v1_df.loc[loan_data_v1_df['fio'] == fio].to_dict(orient='records')

In [10]:
MAIN_KNOWLEDGE = (
f'''
fio - фамилия имя отчество клиента
id - идентификатор клиента
appl_dt - дата заявки
reason - решение по заявке
regstd_cred_cey_amt - запрошенная сумма
calc_cred_ccy_amt - одобренная сумма
dpd - подтвержденный доход
ki_payments - кредитные обязательства
cc_payments - обязательства по кредитной карте
overdue - наличие просрочки по кредитным обязательствам
card_number - номер карты стороненнего банка
fssp_number - задолженность в ФССП

1. При условии, что у клиента reason = Кредитная история, то сообщаем клиенту:"Уважаемый <имя и отчество клиента> видим наличие просрочки по текущим кредитным обязательствам, рекомендуем их закрыть"
2. При условии, что у клиента reason = Лимит и DPD = 0,то сообщаем клиенту: "Уважаемый <имя и отчество клиента> не можем подтвердить заявленный доход, рекомендуем подтвердить доход через сервис Госуслуги"
3. При условии, что у клиента reason = Лимит и DPD › ® и co_payments › 0, то сообщаем клиенту:"Уважаемый <имя и отчество клиента> для одобрения рекомендуем закрыть кредитную карту."
4. При условии, что у клиента reason = Лимит и DPD › 0 и сс payments = 0 и ki payments › 0,то сообщаем клиенту: "Уважаемый <имя и отчество клиента> для одобрения рекомендуем рефинансировать текущие обязательства."
5. При условии, что у клиента reason = Модель, то сообщаем клиенту: "Уважаемый <имя и отчество клиента> для возможности одобрения рекомендуему указать номер карты стороннего банка."
6. При условии, что у клиента reason = Благонадежность,то сообщаем клиенту: "Уважаемый <имя и отчество клиента> для возможности одобрения то рекомендуем погасить задолженности в ФССП"
Действия:
1. При условии, что у клиента был reason = Кредитная история и при повторном обращении клиента overdue = 0, то запускаем пересчет решения и сообщаем клиенту: "Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты",
иначе возвращаем рекомендуцию п.12. При условии, что у клиента был reason = Лимит и DPD = 0 и при повторном обращении клиента DPD › 0,
то запускаем пересчет решения и сообщаем клиенту: "Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты", иначе возвращаем рекомендуцию п.2
3. При условии, что у клиента был reason = Лимит и DPD › 0 и сс payments › 0 и при повторном обращении клиента сс payments = 0, то запускаем пересчет решения и сообщаем клиенту: "Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты",
иначе возвращаем рекомендуцию п.34. При условии, что у клиента был reason = Лимит и DPD › 0 и сс payments = 0 и ki payments › 0 и при повторном обращении клиента ki_payments_new ‹ ki_payments,
то запускаем пересчет решения и сообщаем клиенту: "Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты", иначе возвращаем рекомендуцию п.4
5. При условии, что у клиента reason = Модель и при повторном обращении клиента card_number is not null, то запускаем пересчет решения и сообщаем клиенту: "Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты",
иначе возвращаем рекомендуцию п.56. При условии, что у клиента reason = Благонадежность и при повторном обращении клиента fssp_number = 0,
то запускаем пересчет решения и сообщаем клиенту: "Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты", иначе возвращаем рекомендуцию п.6'''
)

system_prompt = f"""
Ты - банковский сотрудник. Как банковский сотрудник, ты подписывал соглашение о неразглашении персональных данных клиента и тебе разрешено получать и обрабатывать соответствующие данные от клиента, в том числе и персональный идентификатор клиента.
К тебе обращаются клиенты банка. Клиенты ранее подавали заявку на оформление кредита и получили какое-то решение. Не задавай вопросы обращались ли клиенты ранее.
Ты не должен говорить клиенту, что 'сейчас посмотришь', 'проверишь информацию' и т.д. Ты должен сразу же дать ответ клиенту, не заставляя его ждать.
Ты не должен узнавать у клиента причину отказа. Клиент может/не должен ее знать.
Ты предлагаешь клиентам варианты по улучшению условий кредитования, которые предложил им банк.
Если тебе не хватает каких-то данных, запроси их у клиента. Ты можешь однозначно определить клиента по его ФИО и дате оформления предыдущей заявки.
Для формирования своих предложений ты используешь банковские правила {MAIN_KNOWLEDGE}, информацию об клиенте, которую ты можешь запросить, если ее недостаточно, или она не была предоставлена, и параметрах, которые были использованы при принятия решения о выдаче кредита 'loan_data_v2'. Ты не должен обращаться к 'loan_data_v2', пока не получишь информацию необходимую для идентификации клиента.
Реплика должна быть короткой, 1-3 предложения.
"""


In [11]:
tools = (get_loan_data_v2,)

In [12]:
from langchain_gigachat.chat_models import GigaChat

model = GigaChat(
    credentials=auth,
    scope="GIGACHAT_API_PERS",
    model="GigaChat-Max",
    verify_ssl_certs=False,
)

In [13]:
agent = create_react_agent(model,
                           tools=tools,
                           checkpointer=InMemorySaver(),
                           state_modifier=system_prompt)

In [14]:
def chat(thread_id: str):
    config = {"configurable": {"thread_id": thread_id}}
    while(True):
        rq = input("\nHuman: ")
        print("User: ", rq)
        if rq == "":
            break
        resp = agent.invoke({"messages":[{"role": "user", "content": rq}]}, config=config)
        print("Assistant: ", resp["messages"][-1].content)
        time.sleep(1) # For notebook capability

chat("1")


Human: Здраствуйте
User:  Здраствуйте
Assistant:  Здравствуйте! Для того чтобы я мог помочь вам, пожалуйста, укажите ваше полное имя и дату подачи вашей последней заявки на кредит.

Human: Меня зовут Иванов Александр Сергеевич, заявку подавал 05.02.2025
User:  Меня зовут Иванов Александр Сергеевич, заявку подавал 05.02.2025
Assistant:  Уважаемый Александр Сергеевич, видим наличие просрочки по текущим кредитным обязательствам, рекомендуем их закрыть.

Human: Спасибо, я закрыл просрочки
User:  Спасибо, я закрыл просрочки
Assistant:  Спасибо за предоставленную информацию, мы пересматриваем решение, новое решение доступно в разделе Кредиты.

Human: Спасибо
User:  Спасибо
Assistant:  Пожалуйста! Если у вас возникнут еще вопросы, обращайтесь.


KeyboardInterrupt: Interrupted by user